In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


# check the total number of genres and select the represntative genres

In [2]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT genres FROM Korean_book")
check = cur.fetchall()
print(check[0:3])
conn.commit()
conn.close()


[('국내도서,소설,한국소설,역사/대하소설,장르소설,역사소설',), ('국내도서,소설,한국소설,한국소설일반',), ('국내도서,소설,한국소설,한국소설일반',)]


In [3]:
rdd = sc.parallelize(check).map(lambda line: line[0].split(","))
print(rdd.count())
rdd = rdd.flatMap(lambda x: x).map(lambda x: (x,1))
print(rdd.take(5))
print(rdd.count())


44208
[('국내도서', 1), ('소설', 1), ('한국소설', 1), ('역사/대하소설', 1), ('장르소설', 1)]
241726


In [4]:
rdd1 = rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x: -x[1]) # descending
rdd2 = rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1]) # ascending
total_genres_num = rdd1.count()-1
print("total genres:",total_genres_num) # <-- (-1)은 국내도서를 없애기 위해

total genres: 2855


In [5]:
top = 30

for i, value in enumerate(rdd1.collect()):
    if top <= 0:
        break
    print(f"{i}| {value}")
    top-=1

0| ('국내도서', 44208)
1| ('인문', 7693)
2| ('대학교재', 7206)
3| ('경제/경영', 5346)
4| ('외국어', 4285)
5| ('소설', 3074)
6| ('청소년', 3006)
7| ('컴퓨터/IT', 2800)
8| ('시/에세이', 2797)
9| ('기술/공학', 2758)
10| ('예술/대중문화', 2750)
11| ('취미/실용/스포츠', 2539)
12| ('과학', 2385)
13| ('정치/사회', 2275)
14| ('역사/문화', 2254)
15| ('자기계발', 2205)
16| ('요리', 1869)
17| ('건강', 1712)
18| ('종교', 1626)
19| ('컴퓨터', 1454)
20| ('가정/육아', 1076)
21| ('교육학', 1072)
22| ('나라별에세이', 881)
23| ('기술공학', 878)
24| ('인물/자전적에세이', 874)
25| ('고전소설/문학선', 863)
26| ('중/고등참고서', 835)
27| ('장르소설', 819)
28| ('철학', 775)
29| ('인문학일반', 762)


In [8]:
exclued_list = [19,22,23,24,27,29]
representative_genres = []

for i, value in enumerate(rdd1.collect()):
    if i==29:
        break
    if i in exclued_list:
        pass
    else:
        representative_genres.append(value[0])
    top-=1

print(representative_genres)
print(len(representative_genres))

with open("../data/korean-representative-genres.txt", "wb") as f:
    for value in representative_genres:
        f.write((value+"\n").encode("UTF-8"))

['국내도서', '인문', '대학교재', '경제/경영', '외국어', '소설', '청소년', '컴퓨터/IT', '시/에세이', '기술/공학', '예술/대중문화', '취미/실용/스포츠', '과학', '정치/사회', '역사/문화', '자기계발', '요리', '건강', '종교', '가정/육아', '교육학', '고전소설/문학선', '중/고등참고서', '철학']
24


In [30]:
others_genres = []
for i, value in enumerate(rdd1.collect()):
    if i not in genre_idx and i!=0:
        others_genres.append(value[0])

print(others_genres[0:5])
print(len(others_genres)+len(representative_genres))
print(total_genres_num)

with open("../../data/korean-all-genres.txt", "wb") as f:
    for value in others_genres:
        f.write((value+"\n").encode("UTF-8"))

['청소년', '컴퓨터', '교육학', '기술공학', '나라별에세이']
2860
2860


# data cleaning

In [48]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, author, id FROM Korean_book")
data = cur.fetchall()
print(len(data))
conn.commit()
conn.close()


44021


In [3]:
# rdd = sc.parallelize(data)
# rdd = rdd.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)
# print(rdd.take(1))
# print(rdd.count())

In [49]:
## 중복 도서 삭제 확인
rdd = sc.parallelize(data)
S = set({})
# D = set({})
count = 0
real_dup_data = []
for d in rdd.collect():
    if (d[0],d[1]) in S:
        real_dup_data.append(d)
        count += 1
        # D.add(d)
    else:
        S.add((d[0],d[1]))

print(len(S))
# print(len(D))
print(count)
# real_dup_data = list(D)

for dup in real_dup_data:
    print(dup)

44021
0


In [47]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute(f"SELECT title, author, id FROM Korean_book WHERE title==\"오만과 편견\"")
temp = cur.fetchall()
print(temp)

[('오만과 편견', '제인 오스틴', 527)]


In [45]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()

for item in real_dup_data:
    cur.execute(f"DELETE FROM Korean_book WHERE id={item[2]}")
    conn.commit()

cur.close()
conn.close()

In [20]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()

# 하나 빼고 중복되는 도서들 모두 삭제
for i in range(len(real_dup_data)):
    cur.execute(f"SELECT id FROM Korean_book WHERE title=\"{real_dup_data[i][0]}\"")
    same = cur.fetchall()
    print(same)
    del_id = []
    for num, (item) in enumerate(same):
        if num==0:
            pass
        else:
            del_id.append(item)
    print(del_id)
    for id in del_id:
        cur.execute(f"DELETE FROM Korean_book WHERE rowid={id}")
        conn.commit()
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()

conn.close()

[(22420,), (22658,)]
[(22658,)]
[(23138,), (23168,)]
[(23168,)]
[(1069,), (1253,)]
[(1253,)]
[(56152,), (56191,)]
[(56191,)]
[(4382,), (4483,), (4517,)]
[(4483,), (4517,)]
[(569,), (944,)]
[(944,)]
[(2992,), (3029,)]
[(3029,)]
[(513,), (848,)]
[(848,)]
[(2493,), (2546,)]
[(2546,)]
[(2495,), (2511,), (2560,), (2797,), (2836,), (2909,), (2918,)]
[(2511,), (2560,), (2797,), (2836,), (2909,), (2918,)]
[(50408,), (50440,)]
[(50440,)]
[(24423,), (24434,)]
[(24434,)]
[(44467,), (44525,)]
[(44525,)]
[(2015,), (2051,)]
[(2051,)]
[(4675,), (4747,)]
[(4747,)]
[(593,), (994,)]
[(994,)]
[(3897,), (3958,)]
[(3958,)]
[(24677,), (24699,)]
[(24699,)]
[(3903,), (4053,)]
[(4053,)]
[(3141,), (3354,)]
[(3354,)]
[(964,), (11509,)]
[(11509,)]
[(5378,), (5404,), (25427,)]
[(5404,), (25427,)]
[(527,), (603,), (714,)]
[(603,), (714,)]
[(8370,), (8484,)]
[(8484,)]
[(9885,), (10159,)]
[(10159,)]
[(23969,), (24065,)]
[(24065,)]
[(22540,), (22742,)]
[(22742,)]
[(2520,), (2668,), (2739,), (2970,), (53407,)]
[(2668,)

In [ ]:
## 이름은 같지만 에디션, 큰글씨, 세트 등 부가사항들일 붙어있는 도서 삭제
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title, author FROM Korean_book")
data = cur.fetchall()
# print(data[0:3])

dup = {}
count = 0
for id, title, author in data:
    if "(큰글" in title or "(리커버)" in title or "(초판본)" in title\
        or "에디션)" in title or "커버)" in title:
        # cur.execute(f"DELETE FROM Korean_book WHERE rowid=={id}")
        print(id, title, author)
        count +=1
print(count)
conn.commit()
cur.close()
conn.close()

In [ ]:
## 이름은 같지만 에디션, 큰글씨, 세트 등 부가사항들일 붙어있는 도서 삭제
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title, author FROM Korean_book")
data = cur.fetchall()
# print(data[0:3])

dup = {}
for id, title, author in data:
    for i, t, a in data:
        if id!=i and author==a:
            if (title in t) or (t in title):
                dup[author] = ((id,title),(i,t))
            else:
                pass

In [ ]:
print(len(dup))
for author, line in zip(dup.keys(), dup.values()):
    print(author,":",line)

In [12]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
# cur.execute("DELETE FROM Korean_book WHERE title LIKE \"%세트%\"")
cur.execute("SELECT title FROM Korean_book WHERE title LIKE \"%세트%\"")
data = cur.fetchall()
print(len(data))
print(data)
conn.commit()
cur.close()
conn.close()

0
[]


In [114]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
# cur.execute("DELETE FROM Korean_book WHERE id==4315")
cur.execute("SELECT title, id FROM Korean_book WHERE id==4315")
data = cur.fetchall()
print(len(data))
print(data)
conn.commit()
cur.close()
conn.close()

0
[]


In [67]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%에디션%\"")
# cur.execute("UPDATE Korean_book SET name WHERE title LIKE \"%세트%\"")
data = cur.fetchall()
# print(len(data))
# print(data[0])
conn.commit()
cur.close()
conn.close()
print(len(data))

# print(data[0][0])
# title = data[0][0].split("(")[0]
# print(title)
# conn = sqlite3.connect("../Books.db")
# cur = conn.cursor()
# cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={data[0][1]}")
# conn.commit()
# cur.close()
# conn.close()


for i, (title,id,author) in enumerate(data):
    if i==146:
        break
    print(id, title, author)
#     title = title.split("(")[0]
#     conn = sqlite3.connect("../Books.db")
#     cur = conn.cursor()
#     cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={id}")
#     conn.commit()
# cur.close()
# conn.close()

6
4401 열두 개의 달 시화집 일력 에디션 윤동주 외
60489 다이어터 라이트 에디션 1: 결심 편 네온비
60493 다이어터 라이트 에디션 5: 자존감 회복 편 네온비
60500 다이어터 라이트 에디션 6: 유지 편 네온비
60502 다이어터 라이트 에디션 2: 시작 편 네온비
60504 다이어터 라이트 에디션 4: 정체기 편 네온비


In [91]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%리커버%\"")
# cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%달력%\"")
# cur.execute("DELETE FROM Korean_book WHERE id==32427 or id==32431")
# cur.execute("UPDATE Korean_book SET name WHERE title LIKE \"%세트%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

delete_list = []
print(len(data))
for i, item in enumerate(data):
    print(item)
    # if i>3:
    #     # print(item)
    #     delete_list.append(item[1])


for item in data:
    con = sqlite3.connect("../Books.db")
    cur = con.cursor()
    cur.execute(f"DELETE FROM Korean_book WHERE id=={item[1]}")
    con.commit()
    cur.close()
    con.close()


28
('소마(리커버 양장본)', 242, '채사장')
('리버보이(리커버 특별판)', 533, '팀 보울러')
('노인과 바다(영한대역)(문학동네 세계문학전집 200번 출간 기념 리커버 특별판)', 610, '어니스트 헤밍웨이')
('시녀 이야기(리커버 일반판)', 762, '마거릿 애트우드')
('페스트(문학동네  세계문학전집 200번  출간 기념  리커버 특별판)', 2646, '알베르 카뮈')
('작은 것들의 신(문학동네 세계문학전집 10주년 기념 리커버 특별판)', 3566, '아룬다티 로이')
('소송(문학동네  세계문학전집 200번  출간 기념  리커버 특별판)', 4060, '프란츠 카프카')
('데미안+수레바퀴 아래서+싯다르타(초판본)(리커버 한정판)(1900년대 오리지널 초판본 표지디자인)', 4328, '헤르만 헤세')
('사랑하고도 불행한(리커버 특별판)', 4594, '김은비')
('너에게만 좋은 사람이 되고 싶어(리커버 한정판)', 5140, '유귀선')
('작가란 무엇인가 1(헤밍웨이 탄생 123주년 기념 리커버)', 5551, '파리 리뷰')
('개인주의자 선언(30만 부 기념 리커버)', 6918, '문유석')
('거의 모든 사생활의 역사(리커버판)', 7198, '빌 브라이슨')
('몰입의 즐거움(리커버판)', 7389, '미하이 칙센트미하이')
('매우 예민한 사람들을 위한 책(10만 부 기념 리커버)', 7409, '전홍진')
('수학부진아 지도 프로그램 매스리커버리', 8592, '로버트 라이트 외')
('오십에 읽는 논어(교보문고 특별판 리커버)', 9876, '최종엽')
('새로운 미래가 온다(리커버 특별판)', 16450, '다니엘 핑크')
('스티브 니슨의 캔들차트 투자기법(리커버판)', 20602, '스티브 니슨')
('원씽(The One Thing)(리커버 특별판)', 22418, '게리 켈러 외')
('회복탄력성(15만부 기념 리커버)', 22501, '김주환')
('정상에서 만납시다(25주년 리커버판)', 2

In [98]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%리커버%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

delete_list = []
print(len(data))
for i, item in enumerate(data):
    print(item)


# for item in data:
#     title = item[0].split("(")[0]
#     con = sqlite3.connect("../Books.db")
#     cur = con.cursor()
#     cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={item[1]}")
#     con.commit()
#     cur.close()
#     con.close()


0


In [100]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%양장%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

delete_list = []
print(len(data))
for i, item in enumerate(data):
    print(item)


# for item in data:
#     title = item[0].split("(")[0]
#     con = sqlite3.connect("../Books.db")
#     cur = con.cursor()
#     cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={item[1]}")
#     con.commit()
#     cur.close()
#     con.close()


2
('대한민국헌법 양장 포켓북 ', 29466, '대한민국')
('기출탄탄 양장기능사 필기', 31960, '정서경 외')


In [107]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%특별%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

print(len(data))
for i, item in enumerate(data):
    print(item)


# for item in data:
#     title = item[0].split("(")[0]
#     con = sqlite3.connect("../Books.db")
#     cur = con.cursor()
#     cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={item[1]}")
#     con.commit()
#     cur.close()
#     con.close()


37
('젊은작가상 수상작품집 10주년 특별판', 310, '편혜영 외')
('여기는 커스터드, 특별한 도시락을 팝니다', 1007, '가토 겐')
('블레이크씨의 특별한 심리치료법', 3667, '아리엘 도르프만')
('우리들의 특별한 언어', 5917, '김지영 외')
('특별한 아이에서 평범한 아이로', 8902, '권현정 외')
('나의 뇌는 특별하다', 9069, '템플 그랜딘 외')
('[Prestige Lounge 전용 특별 사인본] 오십에 읽는 논어', 9879, '최종엽')
('쿤&포퍼: 과학에는 뭔가 특별한 것이 있다', 14457, '장대익')
('마르크스의 특별한 눈', 14603, '고병권')
('[Prestige Lounge 전용 특별 사인본] 인플레이션에서 살아남기', 20397, '오건영')
('조선시대 사가기록화, 옛 그림에 담긴 조선 양반가의 특별한 순간들', 25539, '박정혜')
('조금 특별한 불교 이야기', 28064, '자현 스님')
('특별행정법', 28993, '김철용 외')
('특별형법', 29470, '이주원')
('주석 특별형법', 29554, '손동권 외')
('형사특별법', 29657, '박상기 외')
('평범한 수집가의 특별한 초대', 30968, '최필규')
('MMCA 이건희컬렉션 특별전 이중섭', 31287, '국립현대미술관')
('특별한 옷장', 32076, '이유미')
('아주 특별한 인연', 33493, '앙브루아즈볼라르')
('특별한 강아지옷 만들기', 46951, '유아연')
('특별한 날의 손뜨개 인형', 47009, '아미구루미닷컴')
('아름답고 특별한 비누 레시피', 47151, '앤마리 파이올라')
('내 아이 200% 채워주는 특별한 유학 + 자녀교육 레시피', 51468, '전영미')
('내게는 특별한 일본어를 부탁해', 52102, '이수경')
('내게는 특별한 프랑스어 어휘를 부탁해', 54206, '이경자')
('내게는 특별한 스페인어 어휘를 부탁해', 54508, '조혜

In [115]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%초판%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

print(len(data))
for i, item in enumerate(data):
    print(item)


# for item in data:
#     title = item[0].split("(")[0]
#     con = sqlite3.connect("../Books.db")
#     cur = con.cursor()
#     cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={item[1]}")
#     con.commit()
#     cur.close()
#     con.close()

1
('김기태의 초판본 이야기', 13020, '김기태')


In [123]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, id, author FROM Korean_book WHERE title LIKE \"%완역%\"")
data = cur.fetchall()
conn.commit()
cur.close()
conn.close()

print(len(data))
for i, item in enumerate(data):
    # if i>16:
    #     break
    print(i, item)

# L = [4,5,6,11,12,13]
# for i, item in enumerate(data):
#     if i in L:
#         title = item[0].split("(")[0]
#         con = sqlite3.connect("../Books.db")
#         cur = con.cursor()
#         cur.execute(f"UPDATE Korean_book SET title=\"{title}\" WHERE id=={item[1]}")
#         con.commit()
# cur.close()
# con.close()

10
0 ('완역 금병매 1', 1639, '소소생')
1 ('동국병감(신완역)(한국고전명저정선 2)', 11018, '김종권 역')
2 ('연해자평 완역정해(상)', 14095, '정진엽')
3 ('완역상주주역', 14317, '임동석')
4 ('펠로폰네소스 전쟁사(상)(완역본)', 25007, '투키디데스')
5 ('난중일기 교주본(신완역)', 25431, '이순신')
6 ('교감완역 난중일기', 25469, '이순신')
7 ('메시지 완역본(영한대역)(가죽장정)', 27649, '유진 피터슨')
8 ('완역집성 정재무도홀기', 48688, '성무경 외')
9 ('동의보감 한글완역본', 59865, '허준')


# pracitice

In [4]:
temp = sc.parallelize([('이재정', '박희윤'), ("김민형", "이재정"), ("이재정", "박희윤")])
temp.take(1)
temp1 = temp.distinct()
for i in temp1.collect():
    print(i)


박희윤
김민형
이상원
이재정


In [58]:
with open("../data/korean-representative-genres.txt", "rb") as f:
    value = f.read().decode("UTF-8").split("\n")
    value = value[:len(value)-1]
    print(value)
    print(len(value))
    

['인문', '대학교재', '경제/경영', '외국어', '소설', '컴퓨터/IT', '시/에세이', '기술/공학', '예술/대중문화', '취미/실용/스포츠', '과학', '역사/문화', '정치/사회', '자기계발', '요리', '건강', '종교', '가정/육아', '고전소설/문학선', '철학']
20
